# Voronoi Infill Generation
 ### Nathan Harding
 Note: I know the plural of index is indices

In [ ]:
from scipy.spatial import Delaunay, Voronoi, voronoi_plot_2d
import matplotlib.pyplot as plt
import matplotlib.tri as mtri
import numpy as np
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import
from stl import mesh
from copy import deepcopy
from random import random

# Constants

In [ ]:
# Maximum number of tries to insert a point
maxAttempts = 1000
# Number of desired points
successfulAttempts = 15

# Point Functions

In [ ]:
#Find all points in the stl
def getPoints(stlMesh):
    allPointsSet = set([tuple(point) for point in stlMesh.v0])
    allPointsSet = allPointsSet.union(set([tuple(point) for point in stlMesh.v1]))
    allPointsSet = allPointsSet.union(set([tuple(point) for point in stlMesh.v2]))
    allPointsSet = list(allPointsSet)
    allPoints = np.array(allPointsSet)
    return allPoints


# Generate a random point within the bounding box of the stl
def generateRandomPoint():
    x = random() * (maxX - minX) + minX
    y = random() * (maxY - minY) + minY
    z = random() * (maxZ - minZ) + minZ
    return np.array([x, y, z])

# Ridge Functions

In [ ]:
# Find the ridges that surround a point
def findRidgeIndexes(point, vor):
    #Finds the point's index
    point_index = np.argmin(np.sum((allPoints - point)**2, axis=1))
    # Finds all ridges containing that point
    return np.where(vor.ridge_points == point_index)[0]


# Find the indexes of vertices a set of ridges refer to
def findRidgeVerticesIndexesFromIndexes(indexes, vor):
    vertices = set()
    for index in indexes:
        for vertex in vor.ridge_vertices[index]:
            vertices.add(vertex)
    return list(vertices)


# Find the Vertices given their indexes
def findRidgeVerticesFromIndexes(indexes, vor):
    return vor.vertices[indexes]


# Given a point, find the vertices belonging to its region
def findVerticesFromRegion(point, vor):
    ridges = findRidgeIndexes(point, vor)
    vertexIndexes = findRidgeVerticesIndexesFromIndexes(ridges, vor)
    return findRidgeVerticesFromIndexes(vertexIndexes, vor)

# Region Functions

In [ ]:
# Find all regions neighboring a given point's region
def findNeighboringRegions(point, vor):
    point_index = np.argmin(np.sum((vor.points - point)**2, axis=1))
    region = vor.point_region[point_index]
    ridges = vor.regions[region]
    neighboringRegions = set()
    for ridge in ridges:
        for i in range(len(vor.regions)):
            if ridge in vor.regions[i] and i != region:
                neighboringRegions.add(i)
    return list(neighboringRegions)


#Gets point indexes of given regions
def getPointsIndexesFromRegions(regions, vor):
    points = []
    for region in regions:
        points.append(np.where(vor.point_region == region)[0][0])
    return points


# Gets points given indexes
def getPointsFromIndexes(indexes, vor):
    return vor.points[indexes]


#Get all the point indexes surrounding a given region
def findNeighboringPoints(point, vor):
    regions = findNeighboringRegions(point,vor)
    indexes = getPointsIndexesFromRegions(regions,vor)
    return getPointsFromIndexes(indexes,vor)

# STL \ Triangle Functions

In [ ]:
# Given a point, find all triangles' indexes from the stl that include it
def findTriangleIndexesFromPoint(point):
    triangles = []
    triangleIndexes = []
    for i in range(len(stlMesh.v1)):
        if stlMesh.v0[i][0] == point[0] and stlMesh.v0[i][1] == point[1] and stlMesh.v0[i][2] == point[2]:
            triangleIndexes.append(i)
        if stlMesh.v1[i][0] == point[0] and stlMesh.v1[i][1] == point[1] and stlMesh.v1[i][2] == point[2]:
            triangleIndexes.append(i)
        if stlMesh.v2[i][0] == point[0] and stlMesh.v2[i][1] == point[1] and stlMesh.v2[i][2] == point[2]:
            triangleIndexes.append(i)
    return triangleIndexes


# Get triangles given their indexes 
def findTrianglesFromIndexes(indexes):
    return stlMesh.points[indexes]
    

# Given a point, find the triangles that it is in
def findTrianglesFromPoint(point):
    indexes = findTriangleIndexesFromPoint(point)
    return findTrianglesFromIndexes(indexes)


# Get normals given their indexes
def findNormalsFromIndexes(indexes):
    return stlMesh.normals[indexes]


# Get normals to triangles containing a point
def findNormalsFromPoint(point):
    indexes = findTriangleIndexesFromPoint(point)
    return findNormalsFromIndexes(indexes)


# Validation Functions

In [ ]:
# Checks if a point is contained inside the triangles containing a given vertex
def isPointContained(point, reference, normals):
    vector = point - reference
    for normal in normals:
        if np.inner(vector,normal) > 0:
            return False
    return True

        
# Checks if a point and its region are inside the shell
def validatePoint(point, vor):
    
    points = findNeighboringPoints(point, vor)
    for p in points:
        if p in boundaryPoints:
            return False
    triangles = findTrianglesFromPoint(points[0])
    pointFlag = False
    for p in points:
        normals = findNormalsFromPoint(p)
        if isPointContained(point,p,normals):
            pointFlag = True
    if not pointFlag:
        return False
    for vertex in findVerticesFromRegion(point,vor):
        vertexFlag = False
        for p in points:
            if p not in shellPoints:
                continue
            normals = findNormalsFromPoint(p)
            if not isPointContained(vertex,p,normals):
                vertexFlag = True
        if not vertexFlag:
            return False
    return True

# Construction Functions

In [ ]:
# Find the normal of 3 points given
def findNormal(x,y,z,point):
    a = np.array(z) - np.array(x)
    b = np.array(y) - np.array(x)
    normal = np.cross(a,b)
    check = np.array(point) - np.array(x)
    if np.inner(check,normal) > 0:
        normal= normal * -1
    return normal / np.linalg.norm(normal)

# Main Program

In [ ]:
# Get STL
#stlMesh = mesh.Mesh.from_file('stl\\cube.stl')
stlMesh = mesh.Mesh.from_file('PATH_TO_FILE')
# Create a list of all unique points
allPoints = getPoints(stlMesh)

# Create a deepcopy of that list to track original shell points
shellPoints = deepcopy(allPoints)

#Find Bounding Coord of model
x = [point[0] for point in allPoints]
y = [point[1] for point in allPoints]
z = [point[2] for point in allPoints]
maxX = max(x)
minX = min(x)
maxY = max(y)
minY = min(y)
maxZ = max(z)
minZ = min(z)
xFac = 10 * max(abs(maxX),abs(minX))
yFac = 10 * max(abs(maxY), abs(minY))
zFac = 10 * max(abs(maxZ), abs(minZ))

# Create a bounding box for the Voronoi
boundaryPoints = np.array([[xFac,0,0], [-xFac,0,0],
                          [0,yFac,0], [0,-yFac,0],
                          [0,0,zFac], [0,0,-zFac],
                          [xFac,yFac,0], [-xFac,yFac,0],
                          [xFac,-yFac,0],[-xFac,-yFac,0],
                          [xFac,0,zFac], [-xFac,0,zFac],
                          [xFac,0,-zFac], [-xFac,0,zFac],
                          [0,yFac,zFac], [0,-yFac,zFac],
                          [0,yFac,-zFac], [0,-yFac,-zFac],
                          [xFac,yFac,zFac], [xFac,yFac,-zFac],
                         [xFac,-yFac,zFac], [xFac,-yFac,-zFac],
                         [-xFac,yFac,zFac], [-xFac,yFac,-zFac],
                         [-xFac,-yFac,zFac], [-xFac,-yFac,-zFac]])
allPoints = np.append(allPoints, boundaryPoints, axis = 0)


att = 0
succ = 0
while att < maxAttempts and succ < successfulAttempts:
    att += 1
    point = generateRandomPoint()
    allPoints = np.append( allPoints,[point], axis = 0)
    vor = Voronoi(allPoints)
    if validatePoint(point, vor):
        succ+=1
    else:
        allPoints = allPoints[:-1]
print("Added "+ str(succ)+" Points")



# Construction

In [ ]:
pointsToAdd = allPoints[len(shellPoints)+len(boundaryPoints):]
pointIndexes = []
for point in pointsToAdd:
    pointIndexes.append(np.argmin(np.sum((vor.points - point)**2, axis=1)))
regionIndexesToAdd = np.array(vor.point_region)[pointIndexes]
    

trianglesToAdd = []

# Loop through all regions
for i in range(len(regionIndexesToAdd)):
    #Grab the ridges that we need to look at
    region = vor.regions[regionIndexesToAdd[i]]
    ridges = []
    for ridge in region:
        ridges.append(vor.ridge_vertices[ridge])
        
    # Find the normal of each ridge
    normals = []
    for ridge in ridges:
        normals.append(findNormal(*vor.vertices[ridge[:3]],pointsToAdd[i]))
    
    #Find the Average normal of each vertex in the region
    vertexNormal = dict()
    vertices = findVerticesFromRegion(pointsToAdd[i], vor)
    vertexNormals = np.zeros((len(vertices),3))
    for j in range(len(ridges)):
        for vertex in ridges[j]:
            if vertex in vertexNormal.keys():
                vertexNormal[vertex] += normals[j]
            else:
                vertexNormal[vertex] = normals[j]
                
    # Normalize the normal of each vertex in the region
    for vertex in vertexNormal.keys():
        vertexNormal[vertex] /= np.linalg.norm(vertexNormal[vertex])
    
    # Shift each vertex slightly away in the opposite direction of its average normal
    alteredVertex = dict()
    for vertex in vertexNormal.keys():
        alteredVertex[vertex] = np.array(vor.vertices[vertex])-(vertexNormal[vertex] / 100)

    #Create triangles from each vertex
    
    for ridge in ridges:
        for j in range(1,len(ridge)-1):
            triangle= []
            triangle.append(alteredVertex[ridge[0]])
            triangle.append(alteredVertex[ridge[j]])
            triangle.append(alteredVertex[ridge[j+1]])
            trianglesToAdd.append(deepcopy(triangle))


# Output

In [ ]:
# Create Mesh for output
outputMesh = np.zeros(len(trianglesToAdd)+len(stlMesh.points), dtype=mesh.Mesh.dtype)

# Add old shell to output
for i in range(len(stlMesh.points)):
    trianglesToAdd.append(np.array([stlMesh.v0[i],stlMesh.v1[i],stlMesh.v2[i]]))

# Add all triangles to new mesh
for i in range(len(trianglesToAdd)):
    outputMesh['vectors'][i] = np.array(trianglesToAdd[i])

# Create Mesh
outputMesh = mesh.Mesh(outputMesh)

# Save Mesh
outputMesh.save('test3.stl')